In [1]:
from dotenv import load_dotenv
load_dotenv(override=True)

True

## Build a Personal Assistant with Subagents

#### 1. Define Tools

Start by defining the tools that require structured inputs. In real applications, these would call actual APIs (Google Calendar, SendGrid, etc.). For this tutorial, you’ll use stubs to demonstrate the pattern.

In [2]:
from langchain.tools import tool

@tool
def create_calendar_event(
    title: str,
    start_time: str,       # ISO format: "2024-01-15T14:00:00"
    end_time: str,         # ISO format: "2024-01-15T15:00:00"
    attendees: list[str],  # email addresses
    location: str = ""
) -> str:
    """
    Create a new calendar event.

    This tool schedules a calendar event with a title, start and end time,
    list of attendees, and an optional location. The event is assumed to be
    created in an external calendar system such as Google Calendar or Outlook.

    Parameters
    ----------
    title : str
        The title or subject of the calendar event.

    start_time : str
        Event start date and time in ISO 8601 format.
        Example: "2024-01-15T14:00:00"

    end_time : str
        Event end date and time in ISO 8601 format.
        Example: "2024-01-15T15:00:00"

    attendees : list[str]
        A list of attendee email addresses to be invited to the event.

    location : str, optional
        The location of the event (physical address or virtual meeting link).
        Defaults to an empty string if not provided.

    Returns
    -------
    str
        A confirmation message indicating that the calendar event
        was successfully created.

    Notes
    -----
    - All date and time values **must** be provided in exact ISO 8601 format.
    - In a real implementation, this function would integrate with
      calendar APIs such as Google Calendar or Microsoft Outlook.
    - This tool is designed for use by LLM agents via LangChain's `@tool`
      decorator.

    Example
    -------
    >>> create_calendar_event(
    ...     title="Project Sync",
    ...     start_time="2024-01-15T14:00:00",
    ...     end_time="2024-01-15T15:00:00",
    ...     attendees=["alice@example.com", "bob@example.com"],
    ...     location="Google Meet"
    ... )
    "Event created: Project Sync from 2024-01-15T14:00:00 to 2024-01-15T15:00:00 with 2 attendees"
    """
    return f"Event created: {title} from {start_time} to {end_time} with {len(attendees)} attendees"

@tool
def send_email(
    to: list[str],      # email addresses
    subject: str,
    body: str,
    cc: list[str] = []
) -> str:
    """
    Send an email message.

    This tool sends an email to one or more recipients with a subject and
    message body. Optional CC recipients may also be included. The email
    is assumed to be sent using an external email delivery service such as
    Gmail, Outlook, or SendGrid.

    Parameters
    ----------
    to : list[str]
        A list of recipient email addresses.

    subject : str
        The subject line of the email.

    body : str
        The main content of the email message.

    cc : list[str], optional
        A list of email addresses to be included as CC recipients.
        Defaults to an empty list if not provided.

    Returns
    -------
    str
        A confirmation message indicating that the email
        was successfully sent.

    Notes
    -----
    - All email addresses must be properly formatted.
    - In a production system, this function would integrate with
      email APIs such as Gmail API, Microsoft Graph, or SendGrid.
    - This tool is designed to be invoked by LLM agents using
      LangChain's `@tool` decorator.

    Example
    -------
    >>> send_email(
    ...     to=["alice@example.com"],
    ...     subject="Meeting Reminder",
    ...     body="Reminder: Our meeting starts at 2 PM.",
    ...     cc=["manager@example.com"]
    ... )
    "Email sent to alice@example.com - Subject: Meeting Reminder"
    """
    return f"Email sent to {', '.join(to)} - Subject: {subject}"

@tool
def get_available_time_slots(
    attendees: list[str],
    date: str,  # ISO format: "2024-01-15"
    duration_minutes: int
) -> list[str]:
    """
    Retrieve available meeting time slots for a group of attendees.

    This tool checks calendar availability for the specified attendees on a
    given date and returns a list of time slots that can accommodate a meeting
    of the requested duration. The availability is assumed to be determined
    by querying one or more external calendar systems.

    Parameters
    ----------
    attendees : list[str]
        A list of attendee email addresses whose calendars should be checked
        for availability.

    date : str
        The date for which availability should be checked, provided in
        ISO 8601 date format.
        Example: "2024-01-15"

    duration_minutes : int
        The desired duration of the meeting in minutes.

    Returns
    -------
    list[str]
        A list of available start times for the meeting on the specified date.
        Each time is represented in 24-hour format (HH:MM).
        Example: ["09:00", "14:00", "16:00"]

    Notes
    -----
    - The date must be provided in valid ISO 8601 format (YYYY-MM-DD).
    - The returned time slots represent start times only and assume sufficient
      availability for the full requested duration.
    - In a production system, this function would integrate with calendar APIs
      such as Google Calendar or Microsoft Outlook.
    - This tool is intended for use by LLM agents via LangChain's `@tool`
      decorator.

    Example
    -------
    >>> get_available_time_slots(
    ...     attendees=["alice@example.com", "bob@example.com"],
    ...     date="2024-01-15",
    ...     duration_minutes=60
    ... )
    ["09:00", "14:00", "16:00"]
    """
    return ["09:00", "14:00", "16:00"]

In [3]:
# Initialize the LLM for use with router / structured output
from langchain.chat_models import init_chat_model
model_gemini_flash = init_chat_model("gemini-2.5-flash", model_provider="google_genai", timeout=30, temperature=0)
model_gemini_flash_lite = init_chat_model("gemini-2.5-flash-lite", model_provider="google_genai", timeout=30, temperature=0)
model_llama_groq = init_chat_model("llama-3.1-8b-instant", model_provider="groq", timeout=30, temperature=0)
model_gpt_4o_mini = init_chat_model("gpt-4o-mini", model_provider="openai", temperature=0)
model_gpt_5_nano = init_chat_model("gpt-5-nano", model_provider="openai", timeout=30, temperature=0)#gpt-4.1
model_gpt_4_dot_1 = init_chat_model("gpt-4.1", model_provider="openai", temperature=0)



#### 2. Create Specialized Subagents

Create a calendar agent

In [5]:
from langchain.agents import create_agent
from utils import get_today_str
from langchain.agents.middleware import HumanInTheLoopMiddleware

CALENDAR_AGENT_PROMPT = (
    "You are a calendar scheduling assistant. "
    "For context today's date is : {today_date} . "
    "Parse natural language scheduling requests (e.g., 'next Tuesday at 2pm') "
    "into proper ISO datetime formats. "
    "Use get_available_time_slots to check availability when needed. "
    "Use create_calendar_event to schedule events. "
    "Always confirm what was scheduled in your final response."
)

calendar_agent = create_agent(
    model=model_gpt_4_dot_1,
    tools=[create_calendar_event, get_available_time_slots],
    system_prompt=CALENDAR_AGENT_PROMPT.format(today_date=get_today_str()),
    middleware=[ 
        HumanInTheLoopMiddleware( 
            interrupt_on={"create_calendar_event": True}, 
            description_prefix="Calendar event pending approval", 
        ), 
    ], 
)

Create an email agent

In [6]:
EMAIL_AGENT_PROMPT = (
    "You are an email assistant. "
    "For context today's date is : {today_date} . "
    "Compose professional emails based on natural language requests. "
    "Extract recipient information and craft appropriate subject lines and body text. "
    "Use send_email to send the message. "
    "Always confirm what was sent in your final response."
)

email_agent = create_agent(
    model=model_gpt_4_dot_1,
    tools=[send_email],
    system_prompt=EMAIL_AGENT_PROMPT.format(today_date=get_today_str()),
    middleware=[ 
        HumanInTheLoopMiddleware( 
            interrupt_on={"send_email": True}, 
            description_prefix="Outbound email pending approval", 
        ), 
    ], 
)

#### 3. Wrap subagents as tools

In [7]:
@tool
def schedule_event(request: str) -> str:
    """Schedule calendar events using natural language.

    Use this when the user wants to create, modify, or check calendar appointments.
    Handles date/time parsing, availability checking, and event creation.

    Input: Natural language scheduling request (e.g., 'meeting with design team
    next Tuesday at 2pm')
    """
    result = calendar_agent.invoke({
        "messages": [{"role": "user", "content": request}]
    })
    return result["messages"][-1].text


@tool
def manage_email(request: str) -> str:
    """Send emails using natural language.

    Use this when the user wants to send notifications, reminders, or any email
    communication. Handles recipient extraction, subject generation, and email
    composition.

    Input: Natural language email request (e.g., 'send them a reminder about
    the meeting')
    """
    result = email_agent.invoke({
        "messages": [{"role": "user", "content": request}]
    })
    return result["messages"][-1].text

#### 4. Create the supervisor agent

In [9]:
from langgraph.checkpoint.memory import InMemorySaver


SUPERVISOR_PROMPT = (
    "You are a helpful personal assistant. "
    "You can schedule calendar events and send emails. "
    "Break down user requests into appropriate tool calls and coordinate the results. "
    "When a request involves multiple actions, use multiple tools in sequence."
)

supervisor_agent = create_agent(
    model=model_gpt_4_dot_1,
    tools=[schedule_event, manage_email],
    system_prompt=SUPERVISOR_PROMPT,
    checkpointer=InMemorySaver(),
)

In [10]:
from uuid import uuid4
query = (
    "Schedule a meeting with the design team next Tuesday at 2pm for 1 hour, "
    "and send them an email reminder about reviewing the new mockups."
)

config = {"configurable": {"thread_id": str(uuid4())}}

interrupts = []
for step in supervisor_agent.stream(
    {"messages": [{"role": "user", "content": query}]},
    config,
):
    for update in step.values():
        if isinstance(update, dict):
            for message in update.get("messages", []):
                message.pretty_print()
        else:
            interrupt_ = update[0]
            interrupts.append(interrupt_)
            print(f"\nINTERRUPTED: {interrupt_.id}")

================================== Ai Message ==================================
Tool Calls:
  schedule_event (call_khjM4e7p1PngimDqSfW2bFYj)
 Call ID: call_khjM4e7p1PngimDqSfW2bFYj
  Args:
    request: Schedule a meeting with the design team next Tuesday at 2pm for 1 hour
  manage_email (call_Ina4MYyGgf7mbs1U2dFuMcag)
 Call ID: call_Ina4MYyGgf7mbs1U2dFuMcag
  Args:
    request: Send an email reminder to the design team about reviewing the new mockups

INTERRUPTED: b40c23d2a28cda63d1b0d61ba2f60a90

INTERRUPTED: 7e491337b27acc0d51764ee25d0ef2b4


In [11]:
for interrupt_ in interrupts:
    for request in interrupt_.value["action_requests"]:
        print(f"INTERRUPTED: {interrupt_.id}")
        print(f"{request['description']}\n")

INTERRUPTED: b40c23d2a28cda63d1b0d61ba2f60a90
Calendar event pending approval

Tool: create_calendar_event
Args: {'title': 'Meeting with Design Team', 'start_time': '2026-01-20T14:00:00', 'end_time': '2026-01-20T15:00:00', 'attendees': [], 'location': ''}

INTERRUPTED: 7e491337b27acc0d51764ee25d0ef2b4
Outbound email pending approval

Tool: send_email
Args: {'to': ['designteam@example.com'], 'subject': 'Reminder: Review of New Mockups Needed', 'body': 'Dear Design Team,\n\nThis is a friendly reminder to review the new mockups at your earliest convenience. Your feedback is important to ensure we stay on track with our project timeline.\n\nPlease let me know if you have any questions or need access to the files.\n\nThank you!\n\nBest regards,\n'}



In [12]:
from langgraph.types import Command 

resume = {}
for interrupt_ in interrupts:
    if interrupt_.id == "2b56f299be313ad8bc689eff02973f16":
        # Edit email
        edited_action = interrupt_.value["action_requests"][0].copy()
        edited_action["arguments"]["subject"] = "Mockups reminder"
        resume[interrupt_.id] = {
            "decisions": [{"type": "edit", "edited_action": edited_action}]
        }
    else:
        resume[interrupt_.id] = {"decisions": [{"type": "approve"}]}

interrupts = []
for step in supervisor_agent.stream(
    Command(resume=resume), 
    config,
):
    for update in step.values():
        if isinstance(update, dict):
            for message in update.get("messages", []):
                message.pretty_print()
        else:
            interrupt_ = update[0]
            interrupts.append(interrupt_)
            print(f"\nINTERRUPTED: {interrupt_.id}")

================================= Tool Message =================================
Name: schedule_event

Your meeting with the design team is scheduled for next Tuesday, January 20, 2026, from 2:00 PM to 3:00 PM. If you would like to add specific attendees or a location, please let me know!
================================= Tool Message =================================
Name: manage_email

The following email was sent to the design team:

Subject: Reminder: Review of New Mockups Needed

Body:
Dear Design Team,

This is a friendly reminder to review the new mockups at your earliest convenience. Your feedback is important to ensure we stay on track with our project timeline.

Please let me know if you have any questions or need access to the files.

Thank you!

Best regards,

Let me know if you need any further changes or additional recipients.
================================== Ai Message ==================================

Your meeting with the design team is scheduled for next Tuesday, 